In [ ]:
%pip install pyodbc
%pip install usaddress


In [ ]:
import pyodbc
import os
import pandas as pd
import re
import usaddress

In [ ]:
os.environ['DW_SERVER'] = "pdsqlirisdw01.hosted.lac.com"
os.environ['DW_DB'] = "PRD_APHIM_UODS_DW"
os.environ['DW_USER'] = "zz_dssteam"
os.environ['DW_PASS'] = "$3rv1se0neF1ve"

In [ ]:
def clean_addr(df, address_var, apt_var, city_var, state_var, zip_var):
    # Replace backslashes with forward slashes in address
    df[address_var] = df[address_var].str.replace(r"\\", "/", regex=True)

    # Extract fractions like '1/2' or '3/4' from the address
    df['addr_frac'] = df[address_var].str.extract(r"(1/2|3/4)", expand=False)
    df[address_var] = df[address_var].str.replace(r"1/2|3/4", "", regex=True).str.strip()

    # Remove unnecessary punctuation and convert to uppercase
    df[address_var] = df[address_var].str.replace(r"[^a-zA-Z0-9\s,]", " ", regex=True).str.upper().str.strip()
    df[apt_var] = df[apt_var].str.replace(r"[^a-zA-Z0-9\s#\-/]", " ", regex=True).str.upper().str.strip()

    # Remove entirely blank entries
    for var in [address_var, apt_var, city_var, state_var, zip_var]:
        df[var] = df[var].replace(r"^\s*$", "", regex=True)

    # Simple regex to extract street (assuming 'number streetname' format)
    street_regex = r"^\d+\s[\w\s]+"
    df['addr'] = df[address_var].apply(lambda x: re.match(street_regex, x).group() if re.match(street_regex, x) else '')

    # Assuming city, state, and zip are already separated correctly
    df['city'] = df[city_var]
    df['state'] = df[state_var]
    df['zip'] = df[zip_var]

    return df

In [ ]:
# Define the keyword lists
invalid_addr1 = "|".join(["UNKNO", "HOMELES", "NEED ADDR", "NEED INFO", "HOMESLESS", "TRANSIENT", "SUPPRESS", "ENCAMPM", 
                          "UNDOMIC", "UNDERPASS", "HOMELESS", "NOADDRESS", "NO ADDRESS", "NONE PROVIDED", "REFUSED", 
                          "123 DEFAULT ST", "PT STATES NONE", "999 TRANSIENT WAY", "UNSHELTERED", "UNHOUSED", "UNKKNOWN", 
                          "UNKNWOWN", "UNKONW", "UNKOWN", "UNSTABLE HOUSING", "XNEEDX", "XNEEDSX", "HOMELEXX", "X NEED X", 
                          "TRANSET", "MISSING", "MAILING ADDRESS", "MAILING ONLY", "NOT AVAILABLE", "NOT PROVIDED", 
                          "THIS IS NOT A HOME ADDRESS", "MAIL RTND BAD ADDRESS", "MAILING ADDRESS ONLY", "NEED NEW ADDRESS", 
                          "FILLING OUT FORMS", "ADDRS VERF", "NO ACTUAL ADDRESS", "NEEDS ADDRESS", "DOES NOT HAVE AN ADDRESS", 
                          "REQUEST CALL", "UNABLE TO PROVIDE", "RETURN MAIL", "BAD ADDRESS", "UNABLE TO LOCATE", 
                          "NOT GIVEN", "NONE GIVEN", "NO KNOWN ADDRESS"])
peh_keywords = invalid_addr1 + "|450 BAUCHET"
peh_keywords1 = ['NOT PROVIDED', 'UNSPECIFIED', '*NONE', 'REFUSED', '`', 'NULL', 'NONE', 'NA', 'N/A',  
                 'ODR', 'SHELTER', 'IN TRANSIT', 'UNK']
peh_keywords2 = peh_keywords1 + ['CA', 'NOT CALIFORNIA', 'LAC']

# Continue defining the keyword lists
invalid_addr15 = "|UPDATE DEMO MAIL RTN|123 TEST|TEST PATIENT|TEST STREET|UPDATE DEMO|ROUTINE POSTPARTUM|" + \
                "123 MISSING|^HOMELES.*|^UNHOUSED.*|^UNSHELTER.*|^TRANSIENT.*|^UNK.*|^PRIV.*|^UNSPEC.*|^PT STATES.*|"
invalid_addr3 = peh_keywords1 + ['YES', '1234 ST', 'TBA', 'TBD', 'TEST 1', 'TEST', 'UNITED STATES', 'US', 'USA',
                                 'MAIL ONLY', 'MAIL RETURNED', 'MAILING', 'ANTHEM BLUE CROSS', 'SANTA MONICA BEACH',
                                 'SPRING', 'TEXT', 'NAME', '1 HOME', 'HOTEL', '1 FAKE', 'SPANISH', 
                                 'SPECIMEN NOT COLLECTED',  'UPDATE', '1 NEED', 'MAIN HOUSE', 'MANAGER', 
                                 'MANAGER OFFICE', 'MANAGER UNIT', 'XX', 'XX OTHER', 'PATIENT REFUSED', 
                                 'NOT PROVIDED', "^REFUS.*", "^NO ADDRESS.*", 'NOT AVAILABLE']
invalid_addr4 = invalid_addr3 + ['CALIFORNIA', 'CA']
invalid_addr5 = invalid_addr4 + ['LOS ANGELES', 'LA', '# LA', '#LA', 'N A', 'NA', '999 TRANSIENT WAY', 
                                 '999 TRANSIENT', '99 JAIL CASE', '123 UNK', '9999 TRANSIENT', '9999 TRANSIENT WAY', 
                                 '9999 TRANSIENT', 'SOUTHBAY ER', 'XXX HOMELESS XXX', 'POST OFFICE BOX', 
                                 'PTIENT STATES', 'SUNSET ED', 'VALLEY PALMS CARE CENTER', 'W LOS ANGELES ED', 
                                 'WLA ED', 'UNK']

# Invalid apartment numbers
invalid_apt1 = invalid_addr1 + "|UNITED CARE FACILITES|HEALTH CARE|WE DO HAVE A NEW REGISTRATION|" + \
               "UNITED CEREBRAL PALSEY|UNITED STATES"
invalid_apt2 = invalid_addr4 + ['YES', '-', '1 LAB', 'APT SUITE FLOOR ETC.', 'NO APT']
invalid_apt3 = invalid_addr5 + ["UNITED CARE FACILITES", "HEALTH CARE", "WE DO HAVE A NEW REGISTRATION", 
                                "UNITED CEREBRAL PALSEY", "UNITED STATES", "BELLFLOWER CHRISTIAN", 
                                "GRAND PARK CONVALESCENT", 'BRUIN PLAZA', 'YES', '-', '1 LAB', 
                                'APT SUITE FLOOR ETC', 'NO APT']

# Key phrases for invalid apartment units
invalid_apt_frag = ["CONV", "HEALTH", "MEDICAL", "NURSING", "ACUTE", "CARE", "SOBER", "REHAB", "ASSISTED",
                    "LIVING", "MANOR", "VILLA", "RETIREMENT", "BEH", "SENIOR", "SNF", "RESID", "SANIT", "CHURCH",
                    "ACUTE", "HEALTHCARE", "WELLNESS", "CONGREGATE", "CONVALESCENT", "GLENDORA GRAND", "REHABILITATION",
                    # Other terms
                    "HOME", "HOUSE", 'HEIGHT', 'COUNTRY', 'GARDEN', 'COTTAGE',  'ADDR', 'C/O', 
                    'POBOX', 'PLAZA', 'UNABLE', 'TRANSIENT', 'JAIL', 'PRISON', 'THE', 'SAME', 'VISTA', 'MAIL', 'PARK', 
                    'OFFICE', 'LAPD', 'LETTER', 'HOTEL', 'INN', 'MOTEL', 'COUNTY', 'UPPER', 'LOWER', 'REAR', 'DOWN', 
                    'UP', 'TEST', 'UNI', 'STADIUM']

# Invalid cities
invalid_city = ['NULL', 'NONE', 'NA', 'N/A', 'YES', 'REFUSED', '`', 'NOT PROVIDED', 'UNK', '', 'UNKKNOWN', 
                'UNKNWOWN', 'UNKONW', 'UNKOWN', 'UNSPECIFIED', 'UNITED STATES', 'US', 'CALIFORNIA', 'CA', ' ', '*NONE']

# POBOX keywords
pobox_keywords = "POBOX|PO BOX|P0 BOX|POSTAL|PO BX|9O BOX|OK BOX|P 0  BOX|P BOX|PB BOX|PI BOX|PIC BOX|P.O. BOX|" + \
                 "P.O.BOX|PIO BOX|PITZER BOX|PO B0X|PO BAX|PO BOS|PO BOT|PO BOV|POX BOX|SHOW BOX|POP BOX|P O BOX|POX BOX"

# Unknown phone numbers
unknown_numbers = ["0000000000", "1111111111", "2222222222", "3333333333", "4444444444", "5555555555", 
                   "6666666666", "7777777777", "8888888888", "9999999999", "0000000001"]

# Unknown email keywords
unknown_emails1 = "GENERATED|NOEMAIL|NOMAIL|REFUSE|UNKNOWN|NO@NO|NA@NA|PATIENT|" + \
                  "DOES NOT|MISSING|NONAME|DECLINED|ASK FOR|NOT GIVEN"
unknown_emails2 = ['NAGMAIL.COM', 'N', 'DECLINED', 'NA', 'NO', 'TEST', 'NONE', 'GOBACKTOASK']

# Unknown names
unknown_names1 = "NO FIRST NAME|NO INFO"
unknown_names2 = ["TEST", "PATIENT"]

In [ ]:
# Assuming invalid_addr1, invalid_addr2, ..., invalid_apt1, ..., etc. are lists
# Convert any string components to lists
invalid_addr1_list=invalid_addr1.split('|')
invalid_addr15_list=invalid_addr15.split('|')
pobox_keywords_list = pobox_keywords.split('|')
invalid_apt1_list = invalid_apt1.split('|')
unknown_emails1_list = unknown_emails1.split('|')
unknown_names1_list = unknown_names1.split('|')

# Combine all keywords into a single list
#all_invalid_keywords = invalid_addr1_list + invalid_addr15_list +invalid_addr3 + invalid_addr4 +\
#invalid_addr5 + invalid_apt1_list + invalid_apt2 + invalid_apt3 + pobox_keywords_list + unknown_numbers +\
#unknown_emails1_list +  unknown_emails2 + unknown_names1_list + unknown_names2

# Combine all keywords into a single list
# Ensure all keywords are strings and in uppercase for case-insensitive comparison
all_invalid_keywords = [str(keyword).upper() for keyword in ( invalid_addr1_list + invalid_addr15_list +invalid_addr3 + invalid_addr4 +\
invalid_addr5 + invalid_apt1_list + invalid_apt2 + invalid_apt3 + pobox_keywords_list + unknown_numbers +\
unknown_emails1_list +  unknown_emails2 + unknown_names1_list + unknown_names2)]

def check_invalid_component(parsed_addr, invalid_keywords):
    for component_type, component_value in parsed_addr.items():
        # Skip certain components from direct keyword matching
        if component_type in ['AddressNumber', 'AddressNumberPrefix', 'AddressNumberSuffix', 'StreetName', 'StreetNamePostType', 
                    'StreetNamePreDirectional','OccupancyType','OccupancyIdentifier','PlaceName','StreetNamePreType']:
            continue

        if any(keyword in component_value.upper() for keyword in invalid_keywords):
            print(f"Invalid component found: {component_type} -> {component_value}")
            return True
    return False

def clean_address(address, invalid_keywords):
    try:
        parsed_address, _ = usaddress.tag(address)
        if check_invalid_component(parsed_address, invalid_keywords):
            return ""  # Return empty string if invalid
        return address  # Return original address if valid
    except usaddress.RepeatedLabelError as e:
        print(f"Error parsing address '{address}': {e}")
        return ""


In [ ]:
#Here we will do the same thing with the city, state and zipcodes i.e. compare them to the invalid keywords

# Define invalid keyword lists for city, state, and zipcode
invalid_city_keywords = invalid_addr4 # Add more as needed
invalid_state_keywords = invalid_addr3  # Add more as needed
invalid_zip_keywords = invalid_addr5  # Add more as needed

def check_invalid_component(parsed_addr, city_keywords, state_keywords, zip_keywords):
    is_invalid = False
    for component_type, component_value in parsed_addr.items():
        if component_type == 'PlaceName' and any(kw in component_value.upper() for kw in city_keywords):
            is_invalid = True
        elif component_type == 'StateName' and any(kw in component_value.upper() for kw in state_keywords):
            is_invalid = True
        elif component_type == 'ZipCode' and any(kw in component_value.upper() for kw in zip_keywords):  # Corrected key
            is_invalid = True
    return is_invalid

def clean_location(city, state, zipcode, city_keywords, state_keywords, zip_keywords):
    try:
        pseudo_address = f"{city}, {state} {zipcode}"  # Corrected variable name
        parsed_address, _ = usaddress.tag(pseudo_address)
        if check_invalid_component(parsed_address, city_keywords, state_keywords, zip_keywords):
            return "", "", ""  # Return empty strings if invalid
        return city, state, zipcode  # Return original values if valid
    except usaddress.RepeatedLabelError:
        print(f"Error parsing pseudo address '{city}, {state} {zipcode}'")
        return "", "", ""  # Handle parsing errors


In [ ]:
#I need to also do the same thing for the apt field
# Define your list of invalid keywords for apartment numbers
invalid_apt_keywords = invalid_addr5  # Add your keywords

def is_valid_apt(apt, invalid_keywords):
    # Check if the apartment number is in the invalid keywords
    if any(kw.lower() in apt.lower() for kw in invalid_keywords):
        return ""
    return apt


In [ ]:
apt_keywords = ['Apt', 'Unit', 'Suite', 'Bldg', 'Room', 'Fl', '#']

# Regular expression pattern for extracting apartment/unit numbers
apt_pattern = r'\b(?:' + '|'.join(apt_keywords) + r')\s*\S+'

def extract_apt(addr):
    match = re.search(apt_pattern, addr, re.IGNORECASE)
    return match.group(0) if match else ""

In [ ]:
def clean_repeat_char(df, var, repeat_num=0):
    # Create a regular expression for repeated characters
    repeat_char_regex = r"^(.)\1*$"

    # Identify rows with invalid data
    df['invalid'] = df[var].apply(lambda x: (bool(re.match(repeat_char_regex, x)) and len(x) > repeat_num) or x in ["0", "00"])

    # Replace invalid data with an empty string
    df[var + '_new'] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

In [ ]:
def clean_spec_char(df, var):
    # Define a regex pattern that checks for the absence of alphanumeric characters
    spec_char_regex = r"^[^a-zA-Z\d]+$"

    # Identify rows with only special characters and mark them as invalid
    df['invalid'] = df[var].apply(lambda x: bool(re.match(spec_char_regex, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

In [ ]:
def clean_num_only(df, var):
    # Remove all non-alphanumeric characters and check if the remaining string is numeric
    df['invalid'] = df[var].apply(lambda x: x.isalnum() and x.isdigit() if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

In [ ]:
def remove_email(df, var):
    # Define a regex pattern for email addresses
    email_regex = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"

    # Identify rows with email addresses and mark them as invalid
    df['invalid'] = df[var].apply(lambda x: bool(re.search(email_regex, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

In [ ]:
def clean_dates_only(df, var):
    # Define regex patterns for date formats
    date_patterns = [
        r"\d{2}/\d{2}/\d{4}",  # mm/dd/yyyy
        r"\d{2}/\d{2}/\d{2}",  # mm/dd/yy
        r"\d{2}-\d{2}-\d{4}",  # mm-dd-yyyy
        r"\d{2}-\d{2}-\d{2}"   # mm-dd-yy
    ]

    # Combine patterns into a single regex
    combined_pattern = '|'.join(date_patterns)

    # Identify rows with date-like strings
    df['invalid'] = df[var].apply(lambda x: bool(re.search(combined_pattern, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

In [ ]:
def clean_two_word(df, var):
    street_suffix = ["RD", "ST", "ROAD", "STREET", "PL", "PLACE", "LN", "LANE", "UPDATE", "OTHER"]
    
    def is_invalid(address):
        if pd.isna(address):
            return False
        words = re.sub(r"[^a-zA-Z0-9\s]", "", address.replace("  ", " ")).split()
        if len(words) != 2:
            return False
        return (words[1] in street_suffix and 
                bool(re.match(r"^(.)\1*$", words[0])) and 
                len(words[0]) > 1) or (words[0].replace("0", "") == "")

    df['invalid'] = df[var].apply(is_invalid)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

In [ ]:
def unit_parse(df, var):
    # List of apartment keywords
    apt_names_st = ['APT', 'APP', 'AP', 'APACE', 'APARTMENT', 'CONDO', 'NO', 'STE', 'SUITE', 'SPC', 'UNIT', 
                    'BUILD', 'BLDG', 'BUILDING', 'RM', 'ROOM']

    # Create regex pattern for extracting unit
    apt_pattern_out = r"(?i)\b(" + "|".join(apt_names_st) + r")\s*\#?\s*[\d\w]+"

    # Function to extract unit
    def extract_unit(address):
        match = re.search(apt_pattern_out, address)
        return match.group(0) if match else ''

    # Function to remove unit
    def remove_unit(address):
        return re.sub(apt_pattern_out, '', address).strip()

    # Apply the functions to create new columns
    df['new_address'] = df[var].apply(remove_unit)
    df['new_address_unit'] = df[var].apply(extract_unit)

    return df
